In [9]:
import os

import pandas as pd

CONFIG_PATH = os.path.join('..', '..', 'named_configs')
HPS = {
	# 'N_EPOCHS': True,  # All the same
	'BATCH_SIZE': True, 'N_SAMPLES': True, 'NOISE_TEMPERATURE': False,
	'LAMBDA': False, 'h_NULL': False, 'h_LR': False, 'f_LR': False,
	'z_RHO': False, 'z_MU': False
}
CONFIGS = {
    'STE_Logistic_84':
        'optuna_ste_best_logistic',
    # 'STE_Gumbel_84':
    #     'optuna_ste_best',
    'STE_Logistic_None':
        'optuna_none_ste_best_logistic',
    # 'STE_Gumbel_None':
    #     'optuna_none_ste_best_gumbel',
    'IMLE_Logistic_66':
        'logistic_candidate_66_test',
    # 'IMLE_Gumbel_66':
    #     'gumbel_candidate_66_test',
    'IMLE_Logistic_None':
        'logistic_candidate_none_1000eps',
    # 'IMLE_Gumbel_None':
    #     'gumbel_candidate_none_1000eps',
    'SFE_Logistic_66':
        'logistic_candidate_66_sfe'
}


def get_quantity(section, name, end_name=','):
    start = section.index(name) + len(name) + 1
    finish = start + section[start: ].index(end_name)
    return section[start: finish]


def latex_float(quantity):
    quantity = float(quantity)
    return f'\\num{{{quantity:.3e}}}'


def latex_int(quantity):
    quantity = int(quantity)
    return f'\\multicolumn{{1}}{{c}}{{{quantity}}}'


def get_hps(config, content, hp_values):
    start_hps = content.index('hyperparameters =')
    content = content[start_hps:]
    hp_values['hp_set'].append(config)
    for hp in HPS:
        quantity = get_quantity(content, f"'{hp}':")
        if '/' in quantity:
            quantity = quantity.split('/')
            quantity = float(quantity[0]) / float(quantity[1])
        if hp == 'LAMBDA':
            quantity = latex_float(quantity) if 'IMLE' in config else ''
        elif (hp == 'BATCH_SIZE') and ('None' in quantity):
            quantity = get_quantity(content, "'LOG2_BATCH_SIZE'")
            quantity = latex_int(2 ** int(quantity))
        else:
            if HPS[hp]:
                quantity = latex_int(quantity)
            else:
                quantity = latex_float(quantity)
        hp_values[hp].append(quantity)

    return hp_values


In [10]:
hp_values = {'hp_set': [], **{hp: [] for hp in HPS}}

for config in CONFIGS:
    with open(os.path.join(CONFIG_PATH, CONFIGS[config] + '.py')) as f:
        content = f.read()
    hp_values = get_hps(config, content, hp_values)


In [11]:
tab = pd.DataFrame(hp_values)
tab = tab.drop(columns='hp_set').transpose()
tab.index = [ '\\hp' + name.replace('_','').lower() for name in tab.index]

In [12]:
print(tab.to_latex(float_format="%.3e", escape=False))

\begin{tabular}{llllll}
\toprule
{} &                       0 &                       1 &                       2 &                       3 &                       4 \\
\midrule
\hpbatchsize        &  \multicolumn{1}{c}{16} &  \multicolumn{1}{c}{16} &  \multicolumn{1}{c}{32} &   \multicolumn{1}{c}{8} &  \multicolumn{1}{c}{32} \\
\hpnsamples         &  \multicolumn{1}{c}{10} &  \multicolumn{1}{c}{50} &  \multicolumn{1}{c}{10} &  \multicolumn{1}{c}{47} &  \multicolumn{1}{c}{10} \\
\hpnoisetemperature &         \num{1.771e-01} &         \num{4.432e-02} &         \num{1.621e-01} &         \num{8.786e-01} &         \num{1.621e-01} \\
\hplambda           &                         &                         &         \num{5.698e+00} &         \num{2.714e+01} &                         \\
\hphnull            &         \num{2.169e-02} &         \num{1.098e-02} &         \num{5.811e-03} &         \num{1.137e-04} &         \num{5.811e-03} \\
\hphlr              &         \num{1.134e-04} &         \

/tmp/ipykernel_164724/1888304596.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tab.to_latex(float_format="%.3e", escape=False))
